# Eingabedaten
Als Eingabedaten dient ein XML-Export aus der CPU HotSpots Ansicht eines JProfiler Snapshots für den Testlauf 245 des Performancetests.

![](ressourcen/jprofiler_jpsnapshot_30-b245.png)

Hierzu wurden die HotSpots selbst als auch die erste Ebene (die "Verursacher" der HotSpots) per "Export View" als XML exportiert.

Anzeige der ersten 10 Zeilen:

In [ ]:
with open (r'input\Hot_Spots_1st_level.xml') as log:
    [print(line[:100] + "...") for line in log.readlines()[:10]]

Die XML-Datei teilt sich in Hotspots und Nodes auf. Beispiel:

```
<hotspot leaf="false" class="javax.servlet.FilterChain" methodName="doFilter" methodSignature="(Ljavax/servlet/ServletRequest;Ljavax/servlet/ServletResponse;)V" time="1574276845" count="4472" lineNumber="-1" percent="29.5">
</pre>
```

Diese enthalten folgende Informationen:

* leaf: Marker für Blatt-Knoten
* class: Name der profilierten Klasse
* methodName: Name der profilierten Methode
* methodSignature: Signatur der profilierten Methode
* time: Die gesamte CPU-Verweildauer als Zeitdauer vom Methodeneinstieg bis zum -ausstieg ("self time") in Mikrosekunden
* count: Gesamtzahl der Methodenaufrufe
* lineNumber: Zeilennummer im ByteCode
* percent: Prozentualer Beitrag der Methode zur gesamten CPU-Verweildauer der Hot-Spot-Messung

XML-Daten als Einzelzeilen einlesen

In [ ]:
import pandas as pd
hotspots = pd.read_csv(
    r'input\Hot_Spots_1st_level.xml', 
    sep="\u0012",
    header=None,
    names=['raw'])
hotspots.head()

Nur die Hotspot- sowie die Node-XML-Elemente auf der ersten Ebene (= Verursacher) sind für die Auswertung interessant und werden folgend verarbeitet.

# Hotspots

Extrahiere Hotspots. Diese sind die <tt>hotspot</tt>-XML-Elemente.

In [ ]:
MARKER_HOTSPOTS = "  <hotspot"
hotspots_marker = hotspots[hotspots['raw'].str.startswith("  <hotspot")]
hotspots_marker.head()

Für diese Auswertung sind nur die Werte aus <tt>class</tt>, <tt>methodName</tt>, <tt>time</tt> und </tt>count</tt> relevant.  

Extrahiere diese Werte und passe deren Datentypen entsprechend an.

In [ ]:
REGEX_XML_ELEMENT=r'^.* class="(?P<class>.*?)" methodName="(?P<method>.*?)".*time="(?P<time>.*?)" count="(?P<count>.*?)".*>$'
hotspots_info = hotspots_marker['raw'].str.extract(REGEX_XML_ELEMENT, expand=True)
hotspots_info['time'] = pd.to_numeric(hotspots_info['time'])
hotspots_info['count'] = pd.to_numeric(hotspots_info['count'])
hotspots_info.head()

Kontrolliere Datentypen

In [ ]:
hotspots_info.info()

# Verursacher (der Hotspots)

Verursacher können über die Node-XML-Elemente der 1. Ebene identifiziert werden.

In [ ]:
MARKER_NODES_1ST_LEVEL = "    <node"
verursacher_marker = hotspots[hotspots['raw'].str.startswith(MARKER_NODES_1ST_LEVEL)]
verursacher_marker.head()

In [ ]:
verursacher_info = verursacher_marker['raw'].str.extract(REGEX_XML_ELEMENT, expand=True)
verursacher_info['time'] = pd.to_numeric(verursacher_info['time'])
verursacher_info['count'] = pd.to_numeric(verursacher_info['count'])
verursacher_info.head()

Kontrolliere Datentypen

In [ ]:
verursacher_info.info()

# Auswertungen


## Vorbereitung: Plot-Funktion

Konfiguriere Visualisierung und erstelle Plot-Funktion zur Wiederverwendung

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

def plotte_daten(series, titel):

    # Styling
    plt.style.use('fivethirtyeight')
    plt.figure(facecolor='white')

    ax = series.plot(
        kind='pie', 
        colors=cm.Spectral(np.linspace(0, 1, len(series))), 
        figsize=(10,10), 
        legend=None, 
        autopct='%1.0f%%',
        pctdistance=1.1, 
        labeldistance=1.2,
        fontsize=10,
        title=titel)
    ax.set_ylabel("")
    fig = ax.get_figure()
    plt.savefig("output/" + titel + ".svg", facecolor=fig.get_facecolor(), bbox_inches="tight")

## Auswertung: Top-10-HotSpots

Summiere die Zeitanteile der Hotspots-Klassen, sortiere absteigend und zeige die TOP-10.

### Ergebnis: Top-10-Hotspots

In [ ]:
top10_hotspots = hotspots_info.groupby(['class']).sum()[['time']].sort_values(by=['time'], ascending=False).head(10)
top10_hotspots

### Ergebnis: Top-10-Hotspot-Plot

In [ ]:
plotte_daten(top10_hotspots['time'], u"Top-10-Hotspot-CPU-Verweildauer in Minuten")

## Auswertung: Top-10-HotSpots relativ zu allen anderen HotSpots ("Andere")

Bestimme alle weiteren Hotspots, die nicht in den Top-10-Hotspots enthalten sind.

In [ ]:
andere_hotspots = hotspots_info[~hotspots_info['class'].isin(top10_hotspots.index)]
andere_hotspots.head()

Setze sprechende Bezeichnung für die anderen Hotspots.

In [ ]:
name_andere = "Andere Hotspots (" + str(len(andere_hotspots)) + " weitere Klassen)"
andere_hotspots_summiert = pd.DataFrame(andere_hotspots.sum(), columns=[name_andere]).T
andere_hotspots_summiert.head()

Füge die Anteile der anderen Hotspots mit den Top-10-Hotspots zusammen.

In [ ]:
ergebnis_top10_hotspots_alle = top10_hotspots.append(andere_hotspots_summiert)
ergebnis_top10_hotspots_alle[['time']]

### Ergebnis: Top-10-Hotspot-Verursacher mit anderen

Erstelle Spalte mit Verweildauer in Minuten und passe Bezeichnung an.

In [ ]:
ergebnis_top10_hotspots_alle['Verweildauer in Minuten'] = ergebnis_top10_hotspots_alle['time'].apply(
    lambda p: '{:.2f}'.format(p/1000/1000/60))
ergebnis_top10_hotspots_alle.index.name="Klassenname"
ergebnis_top10_hotspots_alle[['Verweildauer in Minuten']]

### Ergebnis: Top-10-Hotspots-Plot mit andere

In [ ]:
plotte_daten(ergebnis_top10_hotspots_alle['time'], u'Top-10-Hotspot-CPU-Verweildauer mit Andere in Minuten')

## Auswertung: Top-Verursacher

### Auswertung: Top-10-Hotspot-Verursacher

Summiere die CPU-Verweildauer der Aufrufer-Klassen.

In [ ]:
top10_hotspot_verursacher = verursacher_info.groupby(['class']).sum()[['time']].sort_values(by=['time'], ascending=False).head(10)
top10_hotspot_verursacher

### Ergebnis: Top-10-Verursacher

In [ ]:
plotte_daten(top10_hotspot_verursacher['time'], u"Top-10-Verursacher-CPU-Verweildauer in Minuten")

### Ergebnis: Top-9-Verursacher-Plot gefiltert
Ohne <tt>GlazServletFilter</tt>, da dieser ein Sammelposten für alle anderen Aufrufer sind.

In [ ]:
ergebnis_top10_aufrufer_ohne_glazfilter = top10_hotspot_verursacher[top10_hotspot_verursacher.index != "de.sdvrz.zap.prozesse.baufizap.util.GlazServletFilter"]
plotte_daten(ergebnis_top10_aufrufer_ohne_glazfilter.head(9)['time'], u"TOP-9-Aufrufer-Verweildauer in Minuten (ohne GlazServletFilter)")

### Top-10-HotSpotVerursacher relativ zu allen anderen HotSpots ("Andere")

In [ ]:
andere_hotspot_verursacher = verursacher_info[~verursacher_info.index.isin(top10_hotspot_verursacher.index)]
andere_hotspot_verursacher.head()

Summiere die anderen Aufrufer zusammen

In [ ]:
name_andere = "Aufrufer (" + str(len(andere_hotspot_verursacher)) + " weitere Aufrufer-Klassen)"
andere_verursacher_summiert = pd.DataFrame(andere_hotspot_verursacher.sum(), columns=[name_andere]).T
andere_verursacher_summiert.head()

### Ergebnis: Top-10-Hotspot-Verursacher-Liste

In [ ]:
ergebnis_top10_verursacher_alle = top10_hotspot_verursacher.append(andere_verursacher_summiert)
ergebnis_top10_verursacher_alle['Verweildauer in Minuten'] = ergebnis_top10_verursacher_alle['time'].apply(
    lambda p: '{:.2f}'.format(p/1000/1000/60))
ergebnis_top10_verursacher_alle.index.name="Klassenname"
ergebnis_top10_verursacher_alle[['time', 'Verweildauer in Minuten']]

### Ergebnis: Top-10-Hotspot-Verursacher-Plot inkl. anderer Klassen

In [ ]:
plotte_daten(ergebnis_top10_verursacher_alle['time'], r"Top-10-Hotspot-Verursacher in Minuten")

## Aufschlüsselung der Verursacher nach HotSpots

### Vorverarbeitung: Zusammenführung

Die beiden Daten werden über die Indizies zusammengeführt, wobei nur die relevanten Indizies verwendet werden. Fehlende Daten werden entsprechend aufgefüllt.

In [ ]:
hotspots_analysis = hotspots_info.reindex(
    hotspots_info.index | verursacher_info.index).fillna(method="ffill")
hotspots_analysis.head()

Erstelle Liste mit Zuordnung von Hotspots zu Verursachern

In [ ]:
hotspots_analysis = hotspots_analysis.join(verursacher_info, lsuffix="_hotspot", rsuffix="_aufrufer").dropna()
hotspots_analysis.head()

Erstelle Spalte mit zusammengeführte Klassen- und Methodennamen

In [ ]:
hotspots_analysis['hotspot'] = hotspots_analysis['class_hotspot'] + "#" + hotspots_analysis['method_hotspot']
hotspots_analysis['aufrufer'] = hotspots_analysis['class_aufrufer'] + "#" + hotspots_analysis['method_aufrufer']
hotspots_analysis[['hotspot', 'aufrufer']].head()

Summiere Hotspots und Aufrufer

In [ ]:
group_by_aufrufer_hotspot = hotspots_analysis.groupby(['aufrufer', 'hotspot']).sum().reset_index()
group_by_aufrufer_hotspot.head()

Summiere Aufrufer

In [ ]:
group_by_aufrufer = group_by_aufrufer_hotspot.groupby(['aufrufer']).sum().reset_index()
group_by_aufrufer.head()

Füge Einzeldaten zusammen

In [ ]:
summary_hotspots = pd.merge(group_by_aufrufer_hotspot, group_by_aufrufer, left_on='aufrufer', right_on='aufrufer', suffixes=["", "_gesamt"])
summary_hotspots.sort_values(by=['hotspot' , 'aufrufer', 'count_aufrufer', 'count_aufrufer_gesamt'])
summary_hotspots.head()

In [ ]:
grouped_summary_verursacher = summary_hotspots.groupby(['aufrufer', 'hotspot', 'time_aufrufer_gesamt']).sum()
grouped_summary_verursacher = grouped_summary_verursacher.sort_index(level=2, ascending=False)
grouped_summary_verursacher.head()

### Ergebnis: Aufschlüsselung nach Excel

In [ ]:
grouped_summary_verursacher[['time_hotspot']].to_excel(
    "output/zusammenfassung_aufschluesselung_verursacher_hotspots_nach_verweildauer.xlsx", 
    index_label = ["Verursacher", "Hotspots", "Gesamt-Verweildauer"],
    header=["Verweildauer"],
    sheet_name='Verweildauer nach Zeit')